신경망을 위한 데이터셋 마련하기.
한글 폰트 총 10개 사용.(굴림,새굴림,휴먼아미체,휴먼엑스포체,한양해서,HY견고딕,HY견명조,HY그래픽,HY울릉도B)
한글 데이터셋의 디렉토리에 train이라는 디렉토리를 생성하여 리스트에 있는 단어들을 폰트들과 조합하여 각각의 단어들로 디렉토리를 생성하였고, 최종적으로 각각의 디렉토리에는 각각의 단어별 총 10가지의 폰트들의 bmp파일이 담겨져 있음.

In [ ]:
from PIL import Image, ImageDraw, ImageFont
import os

sentences = [
    "경 기 도 고 양 시 일 산 서 구 탄 현 로 1 3 6 1 1 2 동 1 6 0 1 호",
    "김 동 현",
    "김 재 규 이 경 숙 김 효 섭 박 노 쇠",
    "인 천 광 역 시 연 수 구 아 카 데 미 로 1 1 9",
    "청 춘 이 는 듣 기 만 하 여 도 가 슴 이 설 레 는 말 이 다 너 의 두 손을 가 슴 에 대 고 물 방 아 같 은 심 장 의 고 동 을 들 어 보 라",
    "청 춘 의 피 는 끓 는 다 피 에 뛰 노 는 심 장 은 거 선 의 기 관 같 이 힘 있 다 이 것 이 다 인 류 의 역 사 를 꾸 며 내 려 온 동 력 은 바 로 이 것 이 다",
    "이 성 은 투 명 하 되 얼 음 과 같 으 며 지 혜 는 날 카 로 우 나 갑 속 에 든 칼 이 다 청 춘 의 끓 는 피 가 아 니 더 면 인 간 이 얼 마 나 쓸 쓸 하 랴",
    "얼 음 에 싸 인 만 물 은 죽 음 이 있 을 뿐 이 다"
]

#고유한 글자 추출
unique_chars = set()
for sentence in sentences:
    for char in sentence:
        # 공백 및 특수문자 제외
        if char.strip() and char.isalnum():
            unique_chars.add(char)

print(f"총 고유 글자 수: {len(unique_chars)}")

#이미지 저장 디렉토리 설정
base_output_dir = "c:\\Users\\motus\\OneDrive\\바탕 화면\\인공지능\\인공지능 과제2_글자\\한글글자데이터\\train"
if not os.path.exists(base_output_dir):
    os.makedirs(base_output_dir)

#폰트 파일 경로 설정
#여러 폰트를 리스트로 설정
font_paths = [
    "C:\\Windows\\Fonts\\gulim.ttc",
    "C:\\Windows\\Fonts\\NGULIM.TTF",
    "C:\\Windows\\Fonts\\HMFMPYUN.TTF",
    "C:\\Windows\\Fonts\\HMKMAMI.TTF",
    "C:\\Windows\\Fonts\\HMFMMUEX.TTC",
    "C:\\Windows\\Fonts\\UNI_HSR.TTF",
    "C:\\Windows\\Fonts\\HYGTRE.TTF",
    "C:\\Windows\\Fonts\\HYMJRE.TTF",
    "C:\\Windows\\Fonts\\HYGPRM.TTF",
    "C:\\Windows\\Fonts\\HYWULB.TTF"
]

#글자당 이미지 생성 함수
def create_char_image(char, font_path, save_dir, img_size=(64, 64), font_size=64): 
        # 흰색 배경으로 이미지 생성
        img = Image.new('L', img_size, color=255)
        draw = ImageDraw.Draw(img)
        font = ImageFont.truetype(font_path, size=font_size)
        
        # 글자의 크기 계산
        width, height = draw.textsize(char, font=font)
        
        # 글자를 중앙에 배치
        position = ((img_size[0]-width)/2, (img_size[1]-height)/2)
        draw.text(position, char, fill=0, font=font)
        
        # 폰트 이름 추출 (파일명에서 확장자 제거)
        font_name = os.path.splitext(os.path.basename(font_path))[0]
        
        # 이미지 파일명에 글자와 폰트 이름 포함
        img_filename = f"{char}_{font_name}.bmp"
        img_path = os.path.join(save_dir, img_filename)
        img.save(img_path)
        
#모든 고유 글자에 대해 이미지 생성
for char in unique_chars:
    # 글자별 디렉토리 경로 설정
    char_dir = os.path.join(base_output_dir, char)
    if not os.path.exists(char_dir):
        os.makedirs(char_dir)
    
    # 각폰트별로 이미지 생성 및 저장
    for font_path in font_paths:
        create_char_image(char, font_path, char_dir)


test라는 디렉토리 안에 각기 다른 폰트 3가지를 통하여서 test 데이터 셋을 만듦.

In [ ]:
from PIL import Image, ImageDraw, ImageFont
import os
import numpy as np
import cv2
import random

# 1. 주어진 문장들
sentences = [
    "경 기 도 고 양 시 일 산 서 구 탄 현 로 1 3 6 1 1 2 동 1 6 0 1 호",
    "김 동 현",
    "김 재 규 이 경 숙 김 효 섭 박 노 쇠",
    "인 천 광 역 시 연 수 구 아 카 데 미 로 1 1 9",
    "청 춘 이 는 듣 기 만 하 여 도 가 슴 이 설 레 는 말 이 다 너 의 두 손을 가 슴 에 대 고 물 방 아 같 은 심 장 의 고 동 을 들 어 보 라",
    "청 춘 의 피 는 끓 는 다 피 에 뛰 노 는 심 장 은 거 선 의 기 관 같 이 힘 있 다 이 것 이 다 인 류 의 역 사 를 꾸 며 내 려 온 동 력 은 바 로 이 것 이 다",
    "이 성 은 투 명 하 되 얼 음 과 같 으 며 지 혜 는 날 카 로 우 나 갑 속 에 든 칼 이 다 청 춘 의 끓 는 피 가 아 니 더 면 인 간 이 얼 마 나 쓸 쓸 하 랴",
    "얼 음 에 싸 인 만 물 은 죽 음 이 있 을 뿐 이 다"
]

# 2. 고유한 글자 추출
unique_chars = set()
for sentence in sentences:
    for char in sentence:
        if char.strip() and char.isalnum():  # 공백 및 특수문자 제외
            unique_chars.add(char)

print(f"총 고유 글자 수: {len(unique_chars)}")

# 3. 이미지 저장 디렉토리 설정
base_output_dir = "c:\\Users\\motus\\OneDrive\\바탕 화면\\인공지능\\인공지능 과제2_글자\\한글글자데이터\\test"
if not os.path.exists(base_output_dir):
    os.makedirs(base_output_dir)

# 4. 폰트 경로 설정
font_paths = [
    "C:\\Windows\\Fonts\\ahn_l.ttf",    # 안상수2006가는보통
    "C:\\Windows\\Fonts\\HMFMOLD.TTF", # 휴먼옛체
    "C:\\Windows\\Fonts\\HMFMPYUN.TTF"  # 휴먼편지체
]

# 5. 이미지 생성 및 변형 함수
def create_and_augment_image(char, font_path, output_path):
    # 글자 이미지 생성
    font = ImageFont.truetype(font_path, 64)  # 폰트 크기 설정
    image = Image.new("L", (64, 64), "white")  # 흰 배경의 64x64 이미지 생성
    draw = ImageDraw.Draw(image)
    text_width, text_height = draw.textsize(char, font=font)
    draw.text(((64 - text_width) // 2, (64 - text_height) // 2), char, font=font, fill="black")

    # 변형 적용 함수
    image = augment_image(image)
    
    # 이미지 저장
    image.save(output_path, format="bmp")

# 6. 변형 함수 (가로/세로 노이즈 추가)
def augment_image(image):
    
    transformations = "add_noise"

    for transform in transformations:
        if transform == "add_noise":
            image_cv = np.array(image)
            noise = np.zeros_like(image_cv)

            # 가로 또는 세로 방향 중 하나 선택
            noise_type = random.choice(["horizontal", "vertical"])

            if noise_type == "horizontal":
                for i in range(0, image_cv.shape[0], 5):  # 몇몇 행에만 노이즈 추가
                    noise[i, :] = np.random.normal(0, 10, image_cv.shape[1])
            elif noise_type == "vertical":
                for j in range(0, image_cv.shape[1], 5):  # 몇몇 열에만 노이즈 추가
                    noise[:, j] = np.random.normal(0, 10, image_cv.shape[0])

            image_cv = cv2.add(image_cv, noise.astype(np.uint8))
            image = Image.fromarray(image_cv)

    return image

# 7. 각 글자마다 디렉토리 생성 및 이미지 저장
for char in unique_chars:
    char_dir = os.path.join(base_output_dir, char)
    os.makedirs(char_dir, exist_ok=True)

    # 파일 번호 초기화
    file_index = 1

    for font_path in font_paths:
        output_path = os.path.join(char_dir, f"{file_index}.bmp")
        create_and_augment_image(char, font_path, output_path)
        
        # 인덱스를 증가시켜 다음 이미지 파일 이름으로 사용
        file_index += 1

print("모든 글자 이미지 생성 및 변형 완료.")


위에서 각 글자들을 통해서 bmp파일을 만들었으니 컴퓨터가 이 픽셀 데이터를 읽기 쉽게 각각의 csv파일을 만듦.
합치고 병합하여 train과 test 디렉토리에 각각 merger_data셋을 만듦.

In [ ]:
from PIL import Image
import pandas as pd
import os

# BMP 파일이 저장된 최상위 폴더 경로 설정 (train 폴더 or test 폴더)
train_folder_path = "C:\\Users\\motus\\OneDrive\\바탕 화면\\인공지능\\인공지능 과제2_글자\\한글글자데이터\\test"
merge_data = []

# train, test 폴더 내 하위 폴더를 재귀적으로 탐색
for root, dirs, files in os.walk(train_folder_path):
    # BMP 파일 중 숫자로 된 이름만 정렬하여 가져오기
    bmp_files = sorted(
        [f for f in files if f.endswith(".bmp") and os.path.splitext(f)[0].isdigit()],
        key=lambda x: int(os.path.splitext(x)[0])
    )
    
    if bmp_files:  # 숫자로 된 BMP 파일이 있는 폴더만 처리
        data = []
        for filename in bmp_files:
            # 파일명에서 레이블 추출 (확장자를 제외한 파일명)
            label = os.path.splitext(filename)[0]

            # BMP 파일 열기 및 흑백으로 변환
            file_path = os.path.join(root, filename)
            img = Image.open(file_path).convert('L')  # 흑백 모드로 변환
            pixel_data = list(img.getdata())  # 픽셀 데이터를 리스트로 변환

            # 파일 이름과 픽셀 데이터를 합쳐서 하나의 행으로 만들기
            row = [label] + pixel_data  # 첫 번째 열에 파일 이름(레이블) 추가
            data.append(row)

        # 데이터프레임 생성 (컬럼 이름 없이 파일 이름과 픽셀 정보만 저장)
        df = pd.DataFrame(data)

        # 현재 폴더 이름을 CSV 파일명으로 설정
        folder_name = os.path.basename(root)
        csv_file_path = os.path.join(root, f"{folder_name}.csv")
        df.to_csv(csv_file_path, index=False, header=False)  # 헤더 없이 CSV 파일 저장

        print(f"{csv_file_path} 파일이 성공적으로 생성되었습니다.")
        print(f"CSV 파일 저장 경로: {csv_file_path}")


#파일 삭제하기 
# import os

# # BMP 파일이 저장된 최상위 폴더 경로 설정 (train 폴더 or test 폴더)
# train_folder_path = "C:\\Users\\motus\\OneDrive\\바탕 화면\\인공지능\\인공지능 과제2_글자\\한글글자데이터\\test"

# # train_folder_path 내 하위 폴더를 재귀적으로 탐색하며 '.csv' 파일 삭제
# for root, dirs, files in os.walk(train_folder_path):
#     for file in files:
#         # 파일 이름이 '11.bmp'로 끝나는 경우 삭제
#         if file == ".csv":
#             file_path = os.path.join(root, file)
#             os.remove(file_path)
#             print(f"{file_path} 파일이 삭제되었습니다.")

# 최상위 폴더 경로 설정
train_folder_path = "C:\\Users\\motus\\OneDrive\\바탕 화면\\인공지능\\인공지능 과제2_글자\\한글글자데이터\\test"

# CSV 파일을 저장할 리스트 초기화
csv_files = []

# train 폴더 내 모든 하위 폴더와 파일 순회
for root, dirs, files in os.walk(train_folder_path):
    for file in files:
        if file.endswith(".csv"):
            # CSV 파일의 전체 경로를 리스트에 추가
            csv_files.append(os.path.join(root, file))

# CSV 파일이 있는지 확인하고 병합
if csv_files:
    # 각 CSV 파일을 데이터프레임으로 읽어와서 병합
    merged_data = pd.concat([pd.read_csv(file, header=None) for file in csv_files], ignore_index=True)

    # 병합된 데이터프레임을 하나의 CSV 파일로 저장
    merged_csv_file_path = os.path.join(train_folder_path, "merged_data.csv")
    merged_data.to_csv(merged_csv_file_path, index=False, header=False)

    print(f"모든 CSV 파일이 성공적으로 병합되어 {merged_csv_file_path}에 저장되었습니다.")
else:
    print("병합할 CSV 파일이 없습니다. test 폴더를 확인해 주세요.")







신경망 구성하기
1. 

In [ ]:
import csv
import random
import math
import os
import numpy as np 


In [ ]:
input_nodes = 4096
hidden_nodes = [3000,2000,1000,500,100]
output_nodes = 10

lr = 0.1

n=neuralNetwork(input_nodes,hidden_nodes,output_nodes,lr)


In [ ]:
training_data_file = open("C:\\Users\\motus\\OneDrive\\바탕 화면\\인공지능\\인공지능 과제2_글자\\한글글자데이터\\train\\train_data.csv")
training_data_list = training_data_file.readlines()
training_data_file.close()

ValueError: I/O operation on closed file.

In [4]:
epochs = 10

for e in range(epochs):
    for record in training_data_file:
        all_values = record.split(',')
        inputs =(numpy.asfarray(all_values[1:]) / 255.0 + 0.99) + 0.01
        targets = numpy.zeros(output_nodes) + 0.01
        targets[int(all_values[0])] = 0.99
        n.train(inputs,targets)
    pass
pass


NameError: name 'training_data_file' is not defined